In [1]:
import random
import numpy as np
import copy

In [2]:
def inizializza(sol):
    
    # shake shake shake
    for c in range(0,DIMENSIONE-1):
        sol = tweak(sol)
    return sol


def tweak(sol):
    
    sol_copy = np.copy(sol)
    
    # scegli random due colonne distinte
    x = random.randint(0,DIMENSIONE-1)
    y = random.randint(0,DIMENSIONE-1)
    while x==y:
        y = random.randint(0,DIMENSIONE-1)
        
    # scambia le due colonne
    temp = sol_copy[y]
    sol_copy[y] = sol_copy[x] 
    sol_copy[x] = temp
    
    return sol_copy

In [3]:
def generazione_successori(stato): 
    """"
    genera la lista ordinata di successori di uno stato
    """""
        
    lista = []
    t = len(stato)
    
    for i in range(0, t-1):
        for j in range(i+1, t):
            buffer = copy.deepcopy(stato)
            temp = buffer[i]
            buffer[i] = buffer[j]
            buffer[j] = temp
            eval_successore = eval_stato(buffer)
            lista.append((buffer, eval_successore, (stato[i], stato[j])))  

    lista.sort(key=lambda x: x[1])  # ordiniamo i successori in base alla loro valutazione eval_stato
    return(lista)

In [4]:
def eval_stato(stato):
    
    # definizione della scacchiera N x N
    board  = [[0] * DIMENSIONE for i in range(DIMENSIONE)] 
    
    # inserimento delle regine ('Q') nelle loro posizioni sulla scacchiera
    for i in range(0,DIMENSIONE):
        board[stato[i]][i]='Q'
        
    # spostamenti possibili sulla scacchiera
    dx = [-1,1,-1,1]
    dy = [-1,1,1,-1]
    
    # inizializzazione numero di attacchi (diretti o indiretti)
    conflitti = 0

    for i in range(0,DIMENSIONE):
        x=stato[i]
        y=i
        
        # verifica attacchi sulle diagonali       
        for j in range(0,4):
            tempx = x
            tempy = y
            while (True):
                tempx = tempx + dx[j]           # spostamento sull'asse x
                tempy = tempy + dy[j]           # spostamento sull'asse y
                if ((tempx < 0) or 
                    (tempx >= DIMENSIONE) or
                    (tempy < 0) or 
                    (tempy >= DIMENSIONE)):
                    break                       # si esce dal ciclo while se lo spostamento va fuori la scacchiera
                if (board[tempx][tempy]=='Q'):
                    conflitti = conflitti + 1   # aggiornamento numero di attacchi
    return conflitti

In [5]:
def tabu_test(sequenza, tabu_list):    # è True se una mossa NON è presente nella tabu_list
    
    a, b = sequenza[2]
    if ((a, b) in tabu_list or (b, a) in tabu_list):
        assente = False
    else:
        assente = True
    return(assente)

In [6]:
def stampa(sol):
    
    board = [[0] * DIMENSIONE for i in range(DIMENSIONE)] 

    for x in range(0,DIMENSIONE):
        board[sol[x]][x]='Q'
    print("SCACCHIERA")
    for x in range(0,DIMENSIONE):
        for y in range(0,DIMENSIONE):
            if(board[x][y]=='Q'):
                print("Q   ",end=''),
            else:
                print(".   ",end=''),
        print("\n")
    print("\n\n")

In [7]:
def tabu_search(tabu_tenure):
    
    # impostazione stato iniziale
    stato = list(x for x in range(DIMENSIONE))   
    current = inizializza(stato)
    eval_current = eval_stato(current)
    
    # inizializzazione best
    best = current
    eval_best = eval_current
        
    tabu_list = {}
    neighbours = []
        
    cont = 0
    
    # while not criterio_terminazione():
    while (cont < 100 and eval_best != 0):
            
        # generazione successori (stato, eval_stato, mossa) e ordinamento su eval_stato
        lista_successori = generazione_successori(current)
        if cont == 0:
            l = len(lista_successori)
            print('Numero successori: ', l, '\n')
            
        # selezione successori non tabu
        neighbours = list(filter(lambda n: tabu_test(n, tabu_list), lista_successori))
            
        next_state = neighbours[0][0]         # selezione del migliore dei successori
        eval_next_state = neighbours[0][1]
        print("Iterazione ", cont, ':')
        print('next_state: ', eval_next_state)
        delta = eval_best - eval_next_state
        if delta > 0:
            best = next_state                 # aggiornamento di best
            eval_best = eval_next_state       
        current = next_state
        eval_current = eval_next_state
            
        # decremento del tabu_tenure
        for mossa in tabu_list:
            tabu_list[mossa] = tabu_list[mossa] - 1
                
        # eliminazione elementi con tenure uguale a zero 
        tabu_list = {k: tabu_list[k] for k in tabu_list if tabu_list[k]!=0}   
  
        # inserimento della mossa di next nella tabu_list
        mossa_next = neighbours[0][2]
        tabu_list[mossa_next] = tabu_tenure
        
        print("best_eval =", eval_best,)
        print('mossa:', mossa_next)
        print('tabu_list:', tabu_list, '\n') 
            
        cont += 1
                
    return(best, eval_best)

In [8]:
DIMENSIONE = 10   # dimensione dei lati della scacchiera N x N (dove N è la DIMENSIONE)

In [9]:
soluzione, conflitti = tabu_search(5)

Numero successori:  45 

Iterazione  0 :
next_state:  4
best_eval = 4
mossa: (4, 3)
tabu_list: {(4, 3): 5} 

Iterazione  1 :
next_state:  4
best_eval = 4
mossa: (0, 4)
tabu_list: {(4, 3): 4, (0, 4): 5} 

Iterazione  2 :
next_state:  2
best_eval = 2
mossa: (5, 2)
tabu_list: {(4, 3): 3, (0, 4): 4, (5, 2): 5} 

Iterazione  3 :
next_state:  2
best_eval = 2
mossa: (9, 6)
tabu_list: {(4, 3): 2, (0, 4): 3, (5, 2): 4, (9, 6): 5} 

Iterazione  4 :
next_state:  0
best_eval = 0
mossa: (3, 8)
tabu_list: {(4, 3): 1, (0, 4): 2, (5, 2): 3, (9, 6): 4, (3, 8): 5} 



In [10]:
soluzione

array([4, 6, 8, 2, 7, 1, 3, 5, 0, 9])

In [11]:
conflitti

0

In [12]:
stampa(soluzione)

SCACCHIERA
.   .   .   .   .   .   .   .   Q   .   

.   .   .   .   .   Q   .   .   .   .   

.   .   .   Q   .   .   .   .   .   .   

.   .   .   .   .   .   Q   .   .   .   

Q   .   .   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   Q   .   .   

.   Q   .   .   .   .   .   .   .   .   

.   .   .   .   Q   .   .   .   .   .   

.   .   Q   .   .   .   .   .   .   .   

.   .   .   .   .   .   .   .   .   Q   




